<a href="https://colab.research.google.com/github/JenFaith/DS-Unit-2-Linear-Models/blob/master/Faith__LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [236]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

#I. Wrangle Data

In [237]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv',
                 parse_dates=['SALE DATE'],
                 index_col = 'SALE DATE')


# # Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]


In [238]:
df.dtypes

BOROUGH                             int64
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                         object
dtype: object

In [243]:
# # SALE_PRICE was read as strings.
# # Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .astype(str)
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

df['LAND_SQUARE_FEET'] = (
    df['LAND_SQUARE_FEET']
    .astype(str)
    .str.replace(',','')
    .fillna(0)
    .astype(int)
)
#df['LAND_SQUARE_FEET'] = (df['LAND_SQUARE_FEET'].str.replace(',','').astype(str).astype(int))

##Reformatting Zip Code and Year Built to Integers
df[["ZIP_CODE", "YEAR_BUILT"]] = df[['ZIP_CODE', 'YEAR_BUILT']].fillna(0).astype('int')

# BOROUGH is a numeric column, but arguably should be a categorical feature
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

#Square feet is a measure so Land_Square_Feet should be an integer
# df['LAND_SQUARE_FEET']=df['LAND_SQUARE_FEET'].astype(int)

In [244]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [245]:
# Only use data with one family dwellints as well as sale price more than 100 thousand and less than 2 million.

df = df.query('SALE_PRICE > 100000 and SALE_PRICE < 2000000')
df = (df[df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS']) 


In [246]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                            int64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                    int64
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                          int64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
dtype: object

In [247]:
df.select_dtypes('object').nunique()

BOROUGH                              5
NEIGHBORHOOD                         6
BUILDING_CLASS_CATEGORY              1
TAX_CLASS_AT_PRESENT                 2
BUILDING_CLASS_AT_PRESENT           13
ADDRESS                           3135
APARTMENT_NUMBER                     1
BUILDING_CLASS_AT_TIME_OF_SALE      11
dtype: int64

In [248]:
#Because some columns had a lot of unique categories, I chose to remove them from the dataset
#I also removed apartment number and 'Ease-ment' since it didn't have any values

df.drop(columns = ['ADDRESS',	'APARTMENT_NUMBER','EASE-MENT'], inplace=True)

In [249]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,,,
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,11230,1.0,0.0,1.0,6800,1325.0,1930,1,A9,550000
2019-01-01,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,11427,1.0,0.0,1.0,4000,2001.0,1940,1,A1,200000
2019-01-02,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,A1,10461,1.0,0.0,1.0,3500,2043.0,1925,1,A1,810000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,A1,11226,1.0,0.0,1.0,4000,2680.0,1899,1,A1,125000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,A5,11203,1.0,0.0,1.0,1710,1872.0,1940,1,A5,620000


#II. Split Data

In [250]:
#Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
target = 'SALE_PRICE'
y = df[target]
X = df.drop(columns=target)

mask = X.index <'2019-04-01'

X_train = X[mask]
y_train = y[mask]

X_test = X[~mask]
y_test = y[~mask]

#III. Establish Baseline

In [251]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = [y_train.mean()]*len(y_train)
baseline_MAE = mean_absolute_error(y_train, y_pred)

print("The baseline Mean Absolute Error is:", baseline_MAE)

The baseline Mean Absolute Error is: 214721.52773001452


#IV. Building Linear Regression Model

In [252]:
from sklearn.linear_model import LinearRegression, Ridge
from category_encoders import OneHotEncoder
from sklearn.feature_selection import SelectKBest

One-Hot Encoding of Categorical Features

In [289]:
transformer_ohe = OneHotEncoder(use_cat_names=True)
transformer_skb = SelectKBest(k=38)

# Fit OneHotEncoder to X_train Data
transformer_ohe.fit(X_train)

#First transformation
XT_train = transformer_ohe.transform(X_train)
XT_test = transformer_ohe.transform(X_test)


#Fit Select K Best to X and y data
transformer_skb.fit(XT_train, y_train)

#Transforming data again with SKB
XTT_train = transformer_skb.transform(XT_train)
XTT_test = transformer_skb.transform(XT_test)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 4 11 27 28 36 47] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [290]:
#Linear Regression of Model
model_lr = LinearRegression()
model_lr.fit(XTT_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [286]:
print(model_lr.coef_)
print(model_lr.intercept_)

[ 2.03119208e+14  2.03119208e+14  2.03119207e+14  2.03119207e+14
  2.03119207e+14 -8.40987055e+04  3.21540537e+04 -4.19981741e+05
  2.78280272e+05  1.42634538e+05  5.10150781e+04 -8.90270323e+04
  8.90269188e+04 -2.57696063e+01 -3.60238075e+01  5.09883299e+03
 -2.48053191e+04  3.73121171e+04 -1.38177112e+05  5.36188667e+04
 -1.50404371e+04  8.90272684e+04  1.73937370e+05  5.00353465e+05
 -1.13763617e+02 -8.90270549e+04  2.09697834e+05  1.20670757e+05
  3.39665435e+01  1.63436183e+02  1.36147256e+03  5.09878316e+03
 -5.56128364e+03  2.21721918e+04  3.73121264e+04 -1.38177069e+05
  5.36187861e+04 -1.50403969e+04  8.90275103e+04  1.73937381e+05]
-203119208232448.97


Regularizing Data

In [300]:
#Instructions said to normalize but that made my model worse so I vetoed that

model_r=Ridge(alpha=1.0)
model_r.fit(XT_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

#V. Checking Metrics

Linear Regression

In [291]:
training_MAE = mean_absolute_error(y_train, model_lr.predict(XTT_train))
testing_MAE = mean_absolute_error(y_test, model_lr.predict(XTT_test))
print(training_MAE)
print(testing_MAE)

150261.4478958915
155721.0221273292


In [292]:
#R^2 Score
from sklearn.metrics import r2_score

train_R_SQ = r2_score(y_train, model_lr.predict(XTT_train))
test_R_SQ = r2_score(y_test, model_lr.predict(XTT_test))

print('Training R^2:', train_R_SQ)
print('Test R^2:', test_R_SQ)

Training R^2: 0.46651038011800483
Test R^2: 0.42474671136302367


Ridge Regression

In [301]:
r_training_MAE = mean_absolute_error(y_train, model_r.predict(XT_train))
r_testing_MAE = mean_absolute_error(y_test, model_r.predict(XT_test))
print(r_training_MAE)
print(r_testing_MAE)

150597.36263153714
155590.70384044052


In [302]:
r_train_R_SQ = r2_score(y_train, model_r.predict(XT_train))
r_test_R_SQ = r2_score(y_test, model_r.predict(XT_test))

print('Training R^2:', r_train_R_SQ)
print('Test R^2:', r_test_R_SQ)

Training R^2: 0.4667765442134705
Test R^2: 0.4252141410064396
